In [65]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm import tqdm

import scipy.sparse as sparse
import random
# import implicit
# from implicit.als import AlternatingLeastSquares as ALS

%cd /home/user_3/medistream-recsys/Script
from preprocessing import drop_columns,dict_to_column,dict_to_set,set_to_column,key_to_element

pd.set_option('display.max_rows', 300)
pd.set_option('display.max_columns', 100)

import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)

/home/user_3/medistream-recsys/Script


/home/user_3/anaconda3/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [32]:
df = pd.read_json('/fastcampus-data/select_column_version_3.json')

In [33]:
date_state = "2022-05-12"

# paid orders만 가져오기
df['date_paid'] = pd.to_datetime(df['date_paid'])
df_only_paid = df[~df['date_paid'].isna()]
# 3개월치 데이터만 가져오기
df_date = df_only_paid[df_only_paid['date_paid'] > date_state]
# 취소 안된 것만 가져오기
complete_df = df_date[(df_date['paid'] == True) & (df_date['cancelled']==False)]
# 도서 카테고리만 가져오기
only_book = complete_df[complete_df['name'] == '도서']

# 유저가 중복으로 아이템 구매 삭제
df_duplicated_book = only_book.drop_duplicates(subset=['customer_id','product_ids'])
df_book = df_duplicated_book.sort_values(by='date_paid').reset_index(drop=True)

# train test split

In [38]:
from datetime import datetime, timedelta

date = df_book['date_paid'].max()-timedelta(weeks=3)
train_before_preprocess = df_book[df_book['date_paid'] < date]
test_before_preprocess = df_book[df_book['date_paid'] >= date]

# train 3개 이상 구매 기록유저에 대한 학습 진행
train_drop_row_index = train_before_preprocess['customer_id'].value_counts()[train_before_preprocess['customer_id'].value_counts()>=3].index
train = train_before_preprocess.copy()
train_mf = train_before_preprocess[train_before_preprocess['customer_id'].isin(train_drop_row_index)]
train_ap_mp = train_before_preprocess[~(train_before_preprocess['customer_id'].isin(train_drop_row_index))]

In [35]:
# test 만 있는 item 제거
only_test_items = set(test_before_preprocess.product_ids.unique())-set(train_before_preprocess.product_ids.unique())
test = test_before_preprocess[~test_before_preprocess['product_ids'].isin(only_test_items)]

# train test eda

### 전처리 전후 비교

In [16]:
print('train 전처리 전:',len(train_before_preprocess), 'train 전처리 후:',len(train))

train 전처리 전: 5024 train 전처리 후: 5024


In [17]:
print('test 전처리 전:',len(test_before_preprocess), 'test 전처리 후:',len(test))

test 전처리 전: 886 test 전처리 후: 651


### user 수 비교

In [18]:
print('train 유저 수:',len(train.customer_id.unique()))

train 유저 수: 2323


In [19]:
print('test 유저 수:',len(test.customer_id.unique()))

test 유저 수: 491


### item 개수 비교

In [20]:
print('train 아이템 수 :',len(set(train.product_ids)), 'test 아이템 수 :',len(set(test.product_ids)))

train 아이템 수 : 243 test 아이템 수 : 121


In [21]:
print('train 만 있는 아이템 수:',  len(set(train.product_ids)-set(test.product_ids)))

train 만 있는 아이템 수: 122


In [22]:
print('test 만 있는 아이템 수:', len(set(test.product_ids) - set(train.product_ids)))

test 만 있는 아이템 수: 0


In [29]:
print(train.shape)
print(test.shape)

(5024, 16)
(651, 16)


## train_mf 와 train_ap_mp 비교

In [209]:
print('train_mf 수:',len(train_mf), 'train_ap_mp 수:',len(train_ap_mp))

train_mf 수: 2745 train_ap_mp 수: 2279


In [47]:
print('train_mf 유저 수:',len(train_mf.customer_id.unique()) , 'train_ap_mp 유저 수:',len(train_ap_mp.customer_id.unique()))

train_mf 유저 수: 557 train_ap_mp 유저 수: 1766


In [48]:
print('train_mf 아이템 수:',len(train_mf.product_ids.unique()) , 'train_ap_mp 아이템 수:',len(train_ap_mp.product_ids.unique()))

train_mf 아이템 수: 231 train_ap_mp 아이템 수: 162


# copy kaggle example

# Load Transactions, Reduce Memory

In [41]:
# tmp = train.groupby('customer_id').date_paid.max().reset_index()
# tmp.columns = ['customer_id','max_dat']
# train = train.merge(tmp,on=['customer_id'],how='left')
# train['diff_dat'] = (train.max_dat - train.date_paid).dt.days

# # 다른 날짜를 6보다 작은 것으로 하는건 최신의 정보를 반영하기 위함인가?
# train = train.loc[train['diff_dat']<=6]
# print('Train shape:',train.shape)

Train shape: (3222, 6)


In [ ]:
# train.diff_dat.unique()

# Apriori Transformation:

In [49]:
#Apriori treats duplicates as a single occurence
# 한 유저당 구매한 제품들의 unique -> 중복을 없앰.
train_apri = train.groupby('customer_id')['name_x'].unique().reset_index()
train_apri[:30]

,customer_id,name_x
0,5d60cab24e77525ec5ca13d5,"[흔히보는 정형외과 외래진료 가이드북, 숲을 보는 요통치료]"
1,5d60cb754e77525ec5ca13d9,[흔히보는 정형외과 외래진료 가이드북]
2,5d60cc294e77525ec5ca13e0,[약침의 정석 –통증편]
3,5d60ccd84e77525ec5ca13f2,[증보운곡본초학]
4,5d60cce84e77525ec5ca13f3,[보험한약 브런치 the # 2판 개정판]
5,5d60cd1b4e77525ec5ca13fd,"[동씨침법입문 2판, 윤상훈·권병조의 알짜 근육학]"
6,5d60cd414e77525ec5ca1401,"[흔히보는 정형외과 외래진료 가이드북, 약침의 정석 –통증편, 초음파 유도하 침 시..."
7,5d60cd634e77525ec5ca1402,"[트리거포인트 침치료, 흔히보는 정형외과 외래진료 가이드북, 뇌의학의 첫걸음]"
8,5d60ce694e77525ec5ca140e,[趙紹琴(조소금) 내과학]
9,5d60cea34e77525ec5ca1413,"[흔히보는 정형외과 외래진료 가이드북, 비만문답]"


In [52]:
from mlxtend.preprocessing import TransactionEncoder

In [179]:
transactions = [a[1]['name_x'].tolist() for a in list(train.groupby(['customer_id']))]

In [180]:
te = TransactionEncoder()
te_ary = te.fit(transactions).transform(transactions)

# orders_1hot

In [181]:
orders_1hot = pd.DataFrame(te_ary, columns=te.columns_)
pf = orders_1hot.describe()

In [189]:
f = pf.iloc[0]-pf.iloc[3] # count - frequent
a = f.tolist()
b = list(f.index)
item = pd.DataFrame([[a[r],b[r]]for r in range(len(a))], columns=['Count','Item'])
item = item.sort_values(['Count'], ascending=False).head(10)
print("MOST POPULAR")
item

MOST POPULAR


,Count,Item
84,574,비만문답
211,447,플로차트 정형외과 진단
242,313,흔히보는 정형외과 외래진료 가이드북
144,222,윤상훈·권병조의 알짜 근육학
241,213,황황교수의 임상의를 위한 근거기반 상한금궤 처방 매뉴얼
240,197,황황교수의 개원 한의사를 위한 상한금궤 처방 강의록
131,143,외래에서 꼭 알아야 할 통증증후군 137가지
196,133,카이로프랙틱 기본테크닉론
90,125,사암침의 해석과 임상
126,101,약침의 정석 –통증편


In [190]:
from mlxtend.frequent_patterns import apriori

In [205]:
is_ap = apriori(orders_1hot, min_support=0.09, max_len=None, use_colnames=True)

# Calculate Association Rules

In [207]:
from mlxtend.frequent_patterns import association_rules

In [208]:
rules_confidence = association_rules(is_ap, min_threshold=0.03) # default가 confidence
# rules_lift = association_rules(is_ap, metric= "confidence", min_threshold=0.01)

display(rules_confidence)

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction


# (1) Recommend Most Often Previously Purchased Items

In [62]:
tmp = train.groupby(['customer_id','name_x'])['date_paid'].agg('count').reset_index()
tmp.columns = ['customer_id','name_x','ct']
train = train.merge(tmp,on=['customer_id','name_x'],how='left')
train = train.sort_values(['ct','date_paid'],ascending=False)
train = train.drop_duplicates(['customer_id','name_x'])
train = train.sort_values(['ct','name_x'],ascending=False)
train

/home/user_3/anaconda3/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,_id,date_paid,customer_id,paid,name_x,category_id_y,product_ids,quantity,price,price_total,age_group,한의사 여부,사업자 여부,cancelled,name,slug,ct
4912,63022db558e1680033a577fa,2022-08-21 13:06:29.249000+00:00,5e997e27bfe4260944785d49,True,흔히보는 정형외과 외래진료 가이드북,5cf8bbba0098b2225c5dfaa3,6182113bbc63410023473754,1.0,57000.0,57000.0,20-29,0.0,0.0,False,도서,book,1
4885,63004f04ca740b0024398005,2022-08-20 03:05:17+00:00,60fba9e7d26af90681b1d395,True,흔히보는 정형외과 외래진료 가이드북,5cf8bbba0098b2225c5dfaa3,6182113bbc63410023473754,1.0,57000.0,57000.0,None,0.0,0.0,False,도서,book,1
4882,63002b54ca740b0024397fb0,2022-08-20 00:31:55.091000+00:00,62cb762eb99f01002300b840,True,흔히보는 정형외과 외래진료 가이드북,5cf8bbba0098b2225c5dfaa3,6182113bbc63410023473754,1.0,57000.0,57000.0,30-39,1.0,1.0,False,도서,book,1
4871,62ff4281ca740b0024397d97,2022-08-19 07:58:31.348000+00:00,5dc22ddf2bb59605ca3cf731,True,흔히보는 정형외과 외래진료 가이드북,5cf8bbba0098b2225c5dfaa3,6182113bbc63410023473754,1.0,57000.0,57000.0,20-29,1.0,0.0,False,도서,book,1
4815,62fd8a8eca740b0024397482,2022-08-18 00:41:47.318000+00:00,5d9692e90dabe405b156d97b,True,흔히보는 정형외과 외래진료 가이드북,5cf8bbba0098b2225c5dfaa3,6182113bbc63410023473754,1.0,57000.0,57000.0,40-49,1.0,1.0,False,도서,book,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1017,62a0027f9d93880024072fd2,2022-06-08 02:01:54.409000+00:00,5dcd1a312bb59605ca3cfe72,True,2022 통장잔고 2배 늘려주는 병의원 절세비법,5cf8bbba0098b2225c5dfaa3,6279c9439d93880024062730,1.0,18000.0,18000.0,60-,1.0,0.0,False,도서,book,1
1011,629ff6ef9d93880024072eee,2022-06-08 01:10:52.392000+00:00,5d96906a0dabe405b156d8e3,True,2022 통장잔고 2배 늘려주는 병의원 절세비법,5cf8bbba0098b2225c5dfaa3,6279c9439d93880024062730,1.0,18000.0,18000.0,40-49,1.0,0.0,False,도서,book,1
457,628c77659d9388002406e685,2022-05-24 06:17:08.346000+00:00,5ea8cbe41a3ca36b96784ddf,True,2022 통장잔고 2배 늘려주는 병의원 절세비법,5cf8bbba0098b2225c5dfaa3,6279c9439d93880024062730,1.0,18000.0,18000.0,20-29,1.0,0.0,False,도서,book,1
336,628b0b8e9d9388002406deea,2022-05-23 04:22:30.193000+00:00,5d6696914e77525ec5ca17b9,True,2022 통장잔고 2배 늘려주는 병의원 절세비법,5cf8bbba0098b2225c5dfaa3,6279c9439d93880024062730,1.0,18000.0,18000.0,50-59,1.0,1.0,False,도서,book,1


# (2) Recommend Items Purchased Together

In [73]:
is_ap = apriori(orders_1hot, min_support=0.01, use_colnames=True)
rules_confidence_item = association_rules(is_ap, min_threshold=0.1) # default가 confidence
rules_confidence_item

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
0,(사암침의 해석과 임상),(비만문답),0.033148,0.152214,0.022275,0.672000,4.414829,0.017230,2.584713
1,(비만문답),(사암침의 해석과 임상),0.152214,0.033148,0.022275,0.146341,4.414829,0.017230,1.132598
2,(황황교수의 임상의를 위한 근거기반 상한금궤 처방 매뉴얼),(황황교수의 개원 한의사를 위한 상한금궤 처방 강의록),0.056484,0.052241,0.042694,0.755869,14.468935,0.039743,3.882168
3,(황황교수의 개원 한의사를 위한 상한금궤 처방 강의록),(황황교수의 임상의를 위한 근거기반 상한금궤 처방 매뉴얼),0.052241,0.056484,0.042694,0.817259,14.468935,0.039743,5.163131


In [74]:
apriori_items = rules_confidence_item[['antecedents','consequents']].iloc[:15]

In [115]:
apriori_items['antecedents'] = apriori_items['antecedents'].apply(lambda x: list(map(str,x))[0])
apriori_items['consequents'] = apriori_items['consequents'].apply(lambda x: list(map(str,x))[0])

/home/user_4/anaconda3/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [120]:
apriori_items.rename(columns={"antecedents": "A", "consequents": "A의 연관 아이템 (B)"})

/home/user_4/anaconda3/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,A,A의 연관 아이템 (B)
0,사암침의 해석과 임상,비만문답
1,황황교수의 임상의를 위한 근거기반 상한금궤 처방 매뉴얼,황황교수의 개원 한의사를 위한 상한금궤 처방 강의록
2,황황교수의 개원 한의사를 위한 상한금궤 처방 강의록,황황교수의 임상의를 위한 근거기반 상한금궤 처방 매뉴얼


In [99]:
train_items = train[['date_paid','customer_id','name_x','product_ids']]

# apriori_items
# train_items[train_items['name_x']==]
train_items['apriori_pred'] = train_items['name_x'].apply(lambda x: apriori_items[apriori_items['antecedents']==x]['consequents'].values[0] if x in list(apriori_items['antecedents']) \
                            else None)
# for idx, row in train_items.iterrows():

/home/user_4/anaconda3/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [100]:
name_x_df = train_items.groupby('customer_id')['name_x'].unique().reset_index()
apriori_pred_df  = train_items.groupby('customer_id')['apriori_pred'].unique().reset_index()
real_pred = test.groupby('customer_id')['name_x'].unique().reset_index().rename(columns={'name_x':'real_product'})

train_pred = pd.merge(name_x_df,apriori_pred_df, on='customer_id', how='left')
train_real_pred = pd.merge(train_pred,real_pred, on='customer_id', how='right')

/home/user_4/anaconda3/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [101]:
train_real_pred

/home/user_4/anaconda3/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,customer_id,name_x,apriori_pred,real_product
0,5d61fb634e77525ec5ca1585,[플로차트 정형외과 진단],[None],"[미안침, 파킨슨병 한의진료, 필라테스 해부학(PILATES ANATOMY)]"
1,5d6f735b19efa30eb2914094,[약침의 정석 –통증편],[None],"[뇌의학의 첫걸음, 한의학 상담, 수면장애의 한방치료]"
2,5d83468b3f0e6805c47067c9,[외래에서 꼭 알아야 할 통증증후군 137가지],[None],"[초음파 유도하 침 시술 가이드북, 영어 진료 가이드북 , 비만문답]"
3,5d9690200dabe405b156d8c0,[그린만의 수기의학 원리],[None],"[임상 한의사를 위한 기본 한약처방 강의 2판, 플로차트 정형외과 진단, 장골의 PI 변위는 없다]"
4,5d96fccb0dabe405b156dbc9,[플로차트 정형외과 진단],[None],"[초음파 유도하 침 시술 가이드북, 영어 진료 가이드북 , 카이로프랙틱 기본테크닉론]"
5,5d9708790dabe405b156dccc,"[파킨슨병 한의진료, 이해하면 보이는 어깨치료 ABC, 우리 아이 감기, 비수술정형외과, 무릎통증(아프니까 무릎이다)]",[None],"[감별진단의 정석, 신경학 증상의 감별법, The 정형내과(The Orthopaedic Medicine), 초음파 유도하 침 시술 가이드북, 장골의 PI 변위는 없다]"
6,5d970b4f0dabe405b156dce4,"[SMART 응급진료매뉴얼(세트), SMART 소아진료매뉴얼 3판, SMART 기본 일차진료매뉴얼 3판(세트)]",[None],"[초음파 유도하 침 시술 가이드북, 영어 진료 가이드북 , 소아피부질환해설]"
7,5d9c0beb0dabe405b156e1de,"[이것이 알고싶다! 당뇨병진료, 그림으로 이해하는 수분과 전해질 및 산-염기 대사, 갑상선 진료 완전정복]",[None],"[초음파 유도하 침 시술 가이드북, SMART 응급진료매뉴얼(세트), 영어 진료 가이드북 ]"
8,5dce69512bb59605ca3cffb6,"[한의 피부진료 첫 걸음, 침의 과학적 접근의 이해, 말초신경 약침의학, 근골격계 약침의학]",[None],"[임상 한의사를 위한 기본 한약처방 강의 2판, 일차진료 한의사를 위한 보험한약입문 - 둘째 판, 숲을 보는 요통치료, NEO 인턴 핸드북]"
9,5df191784267e105dffef5d1,[초음파 유도하 침 시술 가이드북],[None],"[비만문답, 바른 자세를 위한 교정운동, 약침의 정석 –통증편]"


In [102]:
# 아이템 맞춘 개수
cnt = 0
for preds, reals in zip(train_real_pred['apriori_pred'], train_real_pred['real_product']):
    for pred in preds:
        if pred in reals:
            cnt += 1
cnt

/home/user_4/anaconda3/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


0